In [12]:
import biosig as bs
import numpy as np
import plotly.graph_objects as go

In [19]:
FILENAME="/Users/lubimyj/Git/experiments/Plansee-Tungstan-W/Denal920/IRCamera/Specimen1/test/Original WFT Files/WAVE0011.WFT"

HDR=bs.header(FILENAME)
data=bs.data(FILENAME)



2000

In [13]:
sr = float(HDR.split('\n')[9][HDR.split('\n')[9].find(':')+2:-1])

In [15]:
def open_data(data):
    res = []
    for item in data:
        res.append(item[0])
    return res
res = open_data(data)

In [17]:
1/500e-9

2000000.0

In [16]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = np.arange(len(res))/sr, y = res))
fig1.show()